# Leaf Classfication 
A For CZ4041 Machine Learning Assignment from PT3 in AY2018/2019 Semester 2.
The group members are:
- LIU Yiqing
- LUO Bingyi
- TENG He Xu
- WANG Jia
- YI Zhiyue
- ZHAO Ziru

## Import necessary libraries and Define Constants

In [2]:
import cv2
import os
import csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

IMAGE_PATH = 'data/images/'
LABEL_PATH = 'data/'
TRAIN_FILE_NAME = 'train.csv'
TEST_FILE_NAME = 'test.csv'
COMMON_HEIGHT_WIDTH = 50


Using TensorFlow backend.


## Load Images and Labels

Load images from the given path

In [204]:
def load_images(path):
    images = os.listdir(path)
    loaded_images = []
    
    for i in range(num):
        loaded_images.append(path + images[i])
    return loaded_images

Resize the images to a uniformed size

In [219]:
def image_resize(file_name, hw):
    image = cv2.imread(file_name, 0)
    new_img = cv2.resize(image, (int(hw), int(hw)))
    return np.reshape(new_img, (int(hw), int(hw), 1))

Load labels from CSV

In [206]:
def load_labels(file_path):
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.reader(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        # Skip the first line
        next(csv_reader)
        # Remove empty lines
        lines = list(line for line in csv_reader if line)
        for line in lines:
            label = {}
            label['id'] = int(line[0])
            label['species'] = line[1]
            labels.append(label)
    
    return labels

In [207]:
def combine_images_and_labels(images, labels):
    id = []
    x = []
    y = []
    for image in images:
        for label in labels:
            if image['id'] == label['id']:
                x.append(image['image'])
                y.append(label['species'])
                id.append( image['id'])
    
    return {
        'id': np.array(id),
        'x': np.array(x),
        'y': np.array(y)
    }

In [208]:
def constructDicitionary(labels):
    uniqueLabels = list(set(list(map(lambda x: x["species"], labels))))
    dictionary = []
    for i in range(len(uniqueLabels)):
        dictionary.append({
            "number": i,
            "text": uniqueLabels[i]
        })
    return dictionary

In [209]:
def encode(dictionary, text):
    for i in range(len(dictionary)):
        if dictionary[i]["text"] == text:
            return dictionary[i]["number"]
    
    return None

In [210]:
def decode(dictionary, number):
    for i in range(len(dictionary)):
        if dictionary[i]["number"] == number:
            return dictionary[i]["text"]
    
    return None

### Execution Scripts

In [220]:
images = load_images(IMAGE_PATH)
train_labels = load_labels(LABEL_PATH + TRAIN_FILE_NAME)
resized_images = []

dictionary = constructDicitionary(train_labels)

for i in range(len(images)):
    resized_image = image_resize(images[i], COMMON_HEIGHT_WIDTH)
    record = {}
    record['id'] = int(images[i].split('.')[0].split('/')[2])
    record['image'] = resized_image
    resized_images.append(record)
    
## for img in resized_images:
##     plt.imshow(img)
##     plt.show()

for j in range(len(train_labels)):
    train_labels[j]["species"] = encode(dictionary, train_labels[j]["species"])

train_data = combine_images_and_labels(resized_images, train_labels)
train_x = train_data['x']
train_y = train_data['y']


In [3]:
print(np.shape(train_y))


NameError: name 'train_y' is not defined

### Construct model

In [272]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, input_shape=(50, 50, 1)))       
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(99, activation=tf.nn.softmax))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_76 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 22, 22, 32)        9248      
_________________________________________________________________
activation_77 (Activation)   (None, 22, 22, 32)        0         
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 9, 9, 64)          18496     
__________

c:\users\lane0\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(50, 50, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\lane0\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  import sys
c:\users\lane0\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by InteractiveShellApp.init_path()


### Train

In [274]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

print(np.shape(train_x))
print(np.shape(train_y))
model.fit(train_x, train_y, epochs=30)

(990, 50, 50, 1)
(990,)
Epoch 1/30
990/990 [==============================] - 3s 3ms/step - loss: 3.3529 - acc: 0.1667
Epoch 2/30
990/990 [==============================] - 2s 2ms/step - loss: 3.1069 - acc: 0.1899
Epoch 3/30
990/990 [==============================] - 2s 2ms/step - loss: 3.0552 - acc: 0.2071
Epoch 4/30
990/990 [==============================] - 2s 2ms/step - loss: 2.8450 - acc: 0.2414
Epoch 5/30
990/990 [==============================] - 2s 2ms/step - loss: 2.7908 - acc: 0.2545
Epoch 6/30
990/990 [==============================] - 2s 2ms/step - loss: 2.7730 - acc: 0.2434
Epoch 7/30
990/990 [==============================] - 2s 2ms/step - loss: 2.6266 - acc: 0.2879
Epoch 8/30
990/990 [==============================] - 2s 2ms/step - loss: 2.4921 - acc: 0.3071
Epoch 9/30
990/990 [==============================] - 2s 2ms/step - loss: 2.4403 - acc: 0.3081
Epoch 10/30
990/990 [==============================] - 2s 2ms/step - loss: 2.3086 - acc: 0.3253
Epoch 11/30
990/990 [====

### Test

In [275]:
test_labels = load_labels(LABEL_PATH + TEST_FILE_NAME)

for k in range(len(test_labels)):
    test_labels[k]["species"] = encode(dictionary, test_labels[k]["species"])

test_data = combine_images_and_labels(resized_images, test_labels)

test_x = test_data['x']
test_y = test_data['y']


In [276]:
output = model.predict(test_x)


    

with open('submission.csv', 'w', newline='') as csvfile:
    fieldnames = ['id']
    
    label_fieldslist = list(set(list(map(lambda x: x["species"], train_labels))))
    for i in range(len(label_fieldslist)):
        fieldnames.append(decode(dictionary, label_fieldslist[i]))
        
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
    for n in range(len(output)):
        #print(np.argmax(output[n]))
        row = {}
        for field in fieldnames:
            species = decode(dictionary, np.argmax(output[n]))
            row["id"] = test_data['id'][n]
            row[species] = 1
            if field not in ["id", species]:
                row[field] = 0
        writer.writerow(row)
